In [ ]:
!pip install cohere tiktoken openai
!pip install langchain

In [ ]:
!pip install python-dotenv

In [ ]:
ls -la

total 20
drwxr-xr-x 1 root root 4096 Feb 13 00:45 ./
drwxr-xr-x 1 root root 4096 Feb 13 00:41 ../
drwxr-xr-x 4 root root 4096 Feb  9 14:20 .config/
-rw-r--r-- 1 root root   68 Feb 13 00:45 .env
drwxr-xr-x 1 root root 4096 Feb  9 14:20 sample_data/


In [ ]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
_ = load_dotenv(find_dotenv())

In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
openai.api_key

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
file_name = "customer_review.csv"
df = pd.read_csv(file_name)

In [ ]:
df.head()

,product_name,customer_review
0,Radiant Glow Highlighting Powder,This powder is awful! It clumps and looks chalky on the skin. Not the radiant glow I was hoping for.
1,LushLips Matte Lipstick in Cherry Bliss,"The color is beautiful, but it's so drying on the lips. Will not be repurchasing."
2,VividEyes Intense Mascara in Jet Black,Clumpy and smudges easily. I looked like a raccoon by the end of the day. Highly disappointed.
3,SkinPerfector BB Cream in Medium,It's okay. Doesn't provide much coverage and feels a bit oily. Good for a light makeup day.
4,ShineNoMore Oil Control Setting Spray,"My makeup stayed put all day! However, it does have a strong chemical smell."


In [ ]:
product_review = df.iloc[51]['customer_review']
print(product_review)

Although the shade is to die for, I wish it wasn't so drying. It's best used with a lip primer or balm.


In [ ]:
# prompt engineering
prompt_template = """
What is the sentiment of the following product review,
which is delimited by triple backticks.

Give your answer using one of the following words: happy, neutral or dissatisfied.
Be concise.
Review text: ```{product_review}```
"""

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
llm_name = "gpt-3.5-turbo"

llm = ChatOpenAI(model=llm_name, temperature=0)

llm_chain = LLMChain(llm=llm, prompt = PromptTemplate.from_template(prompt_template))

In [ ]:
ans = llm_chain.predict(product_review=product_review)

In [ ]:
ans

'dissatisfied'

In [ ]:
df_sub = df.loc[20:40]
results = []

for index, row in df_sub.iterrows():
  # Apply LLM Chain on desired column
  result = llm_chain.predict(product_review = row['product_name'] + row['customer_review'])
  results.append(result)

In [ ]:
results

['dissatisfied',
 'dissatisfied',
 'dissatisfied',
 'neutral',
 'dissatisfied',
 'happy',
 'dissatisfied',
 'neutral',
 'dissatisfied',
 'happy',
 'neutral',
 'dissatisfied',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'happy',
 'neutral']

In [ ]:
df_sub['sentiment'] = results

<ipython-input-24-0c3768931ea4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['sentiment'] = results


In [ ]:
df_sub

,product_name,customer_review,sentiment
20,DreamyDew Hydration Mist with Lavender,The scent is overpowering and gives me a headache. Doesn't feel hydrating either.,dissatisfied
21,LuxLash Volumizing Mascara in Charcoal,It's just okay. Gives volume but dries out quickly and flakes by afternoon.,dissatisfied
22,SilkyMatte Liquid Lip in Coral Dreams,"Loved the color, but it's not transfer-proof. Left marks on my coffee cup.",dissatisfied
23,RefreshMe Under-eye Brightening Cream,"It did reduce puffiness, but no impact on my dark circles. Still searching for the perfect product.",neutral
24,SeamlessCover Full Coverage Concealer,It settled into my fine lines and looked patchy after a few hours. Not impressed.,dissatisfied
25,GoldenHour Bronzer Stick,Easy to apply and blends seamlessly. Perfect for that sun-kissed look!,happy
26,PoreVanish Face Primer,It felt greasy on my skin and didn't hold up my foundation well. I'll pass.,dissatisfied
27,EternalGlow Illuminating Eye Serum,"Lightweight and absorbs quickly. However, the results are not noticeable.",neutral
28,VelvetFinish Blush Palette in Sunset Hues,Colors are beautiful but not long-lasting. They faded within a few hours.,dissatisfied
29,BerryBurst Antioxidant Face Mask,"It tingles a bit, but my skin looked brighter after use. Smells delicious too!",happy
